In [ ]:
%pip install -r ./requirements.txt

In [ ]:
"""
Dataset download from Google Drive using gdown
(Uncomment the following code to download the dataset from Google Drive using gdown)
"""
# import gdown

# !mkdir dataset
# file_id = "175dbrMckA7WGtLXeT55xx06hsDSkRJiR"
# output = "dataset/dataset.zip"
# gdown.download(id=file_id, output=output, quiet=False)

# !unzip ./dataset/dataset.zip -d ./dataset

# Import Modules

In [54]:
import random
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
import copy

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from torch_geometric.data import Data, DataLoader

from Graph import GraphSage, GAT

In [4]:
SEED = 42
deterministic = True

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn

# Data Pre-processing

In [252]:
# Load pokemon.csv and combats.csv
pokemon_data = pd.read_csv('./dataset/pokemon.csv')
combats_data = pd.read_csv('./dataset/combats.csv')

# Encode vertex values to unique integers
label_encoder = LabelEncoder()
pokemon_data['#'] = label_encoder.fit_transform(pokemon_data['#'])
combats_data[['First_pokemon', 'Second_pokemon', 'Winner']] = \
    combats_data[['First_pokemon', 'Second_pokemon', 'Winner']].apply(label_encoder.transform)

features_to_normalize = ['HP', 'Attack', 'Defense', 'Speed', 'Generation', 'Sp. Atk', 'Sp. Def']
features_else = ['Type 1_Bug', 'Type 1_Dark', 'Type 1_Dragon', 'Type 1_Electric',
              'Type 1_Fairy', 'Type 1_Fighting', 'Type 1_Fire', 'Type 1_Flying',
              'Type 1_Ghost', 'Type 1_Grass', 'Type 1_Ground', 'Type 1_Ice',
              'Type 1_Normal', 'Type 1_Poison', 'Type 1_Psychic', 'Type 1_Rock',
              'Type 1_Steel', 'Type 1_Water', 'Type 2_Bug', 'Type 2_Dark',
              'Type 2_Dragon', 'Type 2_Electric', 'Type 2_Fairy', 'Type 2_Fighting',
              'Type 2_Fire', 'Type 2_Flying', 'Type 2_Ghost', 'Type 2_Grass',
              'Type 2_Ground', 'Type 2_Ice', 'Type 2_Normal', 'Type 2_Poison',
              'Type 2_Psychic', 'Type 2_Rock', 'Type 2_Steel', 'Type 2_Water']

scaler = MinMaxScaler()

pokemon_data[features_to_normalize] = \
    scaler.fit_transform(pokemon_data[features_to_normalize])


pokemon_data = pd.get_dummies(pokemon_data, columns=['Type 1', 'Type 2'])
pokemon_data[features_else] = \
    pokemon_data[features_else].astype(int)

hp_weight = 5.0
pokemon_data['HP'] *= hp_weight

pokemon_data['Legendary'] = pokemon_data['Legendary'].astype(int)
pokemon_data['total_stat'] = pokemon_data[features_to_normalize].sum(axis=1)

In [ ]:
# Split combats data into train and test
train_combats, test_combats = train_test_split(combats_data, test_size=0.2, random_state=SEED)

# Extract unique vertex values from train_combats and test_combats
train_vertices = set(train_combats['First_pokemon']).union(set(train_combats['Second_pokemon']))
test_vertices = set(test_combats['First_pokemon']).union(set(test_combats['Second_pokemon']))

# Split pokemon data into train and test based on the vertices
train_pokemon = pokemon_data[pokemon_data['#'].isin(train_vertices)]
test_pokemon = pokemon_data[pokemon_data['#'].isin(test_vertices)]

# Decode vertex values back to original values if needed
train_pokemon['#'] = label_encoder.inverse_transform(train_pokemon['#'])
test_pokemon['#'] = label_encoder.inverse_transform(test_pokemon['#'])

# Set "#" as index for train_pokemon and test_pokemon dataframes
train_pokemon.set_index('#', inplace=True)
test_pokemon.set_index('#', inplace=True)

In [ ]:
features = ['total_stat']+features_to_normalize + features_else

X_data = pokemon_data[features].values
X_data = torch.tensor(X_data, dtype=torch.float)

# for train dataset
X_train = train_pokemon[features].values
X_train = torch.tensor(X_train, dtype=torch.float)
edges_train = []
neg_edge_index_train = []

for _, row in train_combats.iterrows():
    first_pokemon = row['First_pokemon']
    second_pokemon = row['Second_pokemon']
    winner = row['Winner']

    if first_pokemon == winner:
      edges_train.append((second_pokemon, first_pokemon))
      neg_edge_index_train.append((first_pokemon, second_pokemon))

    else:
      edges_train.append((first_pokemon, second_pokemon))
      neg_edge_index_train.append((second_pokemon, first_pokemon))

neg_edge_index_train = torch.tensor(neg_edge_index_train, dtype=torch.long).t()
edge_index_train = torch.tensor(edges_train, dtype=torch.long).t()

data_train = Data(x=X_data, edge_index=edge_index_train, neg_edge_index = neg_edge_index_train)
train_loader = DataLoader([data_train], batch_size=1, shuffle=True)

# for test dataset
X_test = test_pokemon[features].values
X_test = torch.tensor(X_test, dtype=torch.float)
edges_test = []
neg_edge_index_test = []

for _, row in test_combats.iterrows():
    first_pokemon = row['First_pokemon']
    second_pokemon = row['Second_pokemon']
    winner = row['Winner']

    if first_pokemon == winner:
      edges_test.append((second_pokemon, first_pokemon))
      neg_edge_index_test.append((first_pokemon, second_pokemon))

    else:
      edges_test.append((first_pokemon, second_pokemon))
      neg_edge_index_test.append((second_pokemon, first_pokemon))

edge_index_test = torch.tensor(edges_test, dtype=torch.long).t()
neg_edge_index_test = torch.tensor(neg_edge_index_test, dtype=torch.long).t()

data_test = Data(x=X_data, edge_index=edge_index_test, neg_edge_index= neg_edge_index_test)
test_loader = DataLoader([data_test], batch_size=1, shuffle=False)

# for total data
edge = edges_train
edge.extend(edges_test)
edge_index = torch.tensor(edge, dtype=torch.long).t()

data_total = Data(x=X_data, edge_index = edge_index)

# Define Train, Test, Predict Frameworks

In [255]:
def train(model, optimizer, train_loader, device):
    model.train()
    total_loss = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)

        pos_edge_index = data.edge_index
        neg_edge_index = data.neg_edge_index

        pos_head = out[pos_edge_index[0]]
        pos_tail = out[pos_edge_index[1]]
        neg_head = out[neg_edge_index[0]]
        neg_tail = out[neg_edge_index[1]]

        pos_pred = model.predict(pos_head, pos_tail)
        neg_pred = model.predict(neg_head, neg_tail)

        pos_loss = F.binary_cross_entropy_with_logits(pos_pred, torch.ones_like(pos_pred))
        neg_loss = F.binary_cross_entropy_with_logits(neg_pred, torch.zeros_like(neg_pred))
        loss = pos_loss + neg_loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(train_loader)

def test(model, loader, device):
    model.eval()
    auc = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index)

            pos_edge_index = data.edge_index
            neg_edge_index = data.neg_edge_index

            pos_head = out[pos_edge_index[0]]
            pos_tail = out[pos_edge_index[1]]
            neg_head = out[neg_edge_index[0]]
            neg_tail = out[neg_edge_index[1]]

            pos_pred = model.predict(pos_head, pos_tail)
            neg_pred = model.predict(neg_head, neg_tail)
            preds = torch.cat([pos_pred, neg_pred])

            pos_labels = torch.ones_like(pos_pred)
            neg_labels = torch.zeros_like(neg_pred)
            labels = torch.cat([pos_labels, neg_labels])

            auc += roc_auc_score(labels.cpu(), preds.cpu())

    return auc / len(loader)

def predict(model, head, tail, data, device):
    model.eval()
    with torch.no_grad():
        data = data.to(device)
        out = model(data.x, data.edge_index)
        head_feature = out[head]
        tail_feature = out[tail]
        score = model.predict(head_feature, tail_feature)
        reverse_score = model.predict(tail_feature, head_feature)
        return torch.sigmoid(score).item(), torch.sigmoid(reverse_score).item()

# Train

In [256]:
# Choose model from ["GCN", "GraphSage_mean", "GraphSage_maxpool", "GAT"]
def train_model(epoch, model_type, data_train, data_test, train_loader, test_loader, device):
    EPOCH = epoch
    MODEL = model_type

    if MODEL == "GCN":
        model = GraphSage(num_layers=2, dim_in=X_data.shape[1], dim_hidden=64, dim_out=8, agg_type="gcn")
    elif MODEL == "GraphSage_mean":
        model = GraphSage(num_layers=2, dim_in=X_data.shape[1], dim_hidden=64, dim_out=8, agg_type="mean")
    elif MODEL == "GraphSage_maxpool":
        model = GraphSage(num_layers=2, dim_in=X_data.shape[1], dim_hidden=64, dim_out=8, agg_type="maxpool")
    elif MODEL == "GAT":
        model = GAT(dim_in=X_data.shape[1], dim_hidden=64, dim_out=8, dropout = 0.5, alpha = 0.2, num_heads = 8)

    model = model.to(device)

    data_train = data_train.to(device)
    data_test = data_test.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=5e-4)

    patience = 3 # for early stopping
    best_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(EPOCH):
        loss = train(model, optimizer, train_loader, device)
        if epoch % 10 == 0:
            test_auc = test(model, test_loader, device)
            train_auc = test(model, train_loader, device)
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train AUC: {train_auc:.4f}, Test AUC: {test_auc:.4f}')
            if loss < best_loss:
                best_loss = loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve == patience:
                    print("Early stopping triggered")
                    break
    return model

# calculate prediction accuracy
def test_model(model, data_total, data_test, device):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    head_list, tail_list = data_test.edge_index
    head_list = list(head_list)
    tail_list = list(tail_list)

    pos = 0
    for idx in range(len(head_list)):
        head = head_list[idx]
        tail = tail_list[idx]
        prediction, reverse_prediction = predict(model, head, tail, data_total, device)
        if prediction > reverse_prediction:
            pos += 1

    print(f"Prediction accuracy: {pos/len(head_list)}")

In [257]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# select model from model_types
model_types = ["GCN", "GraphSage_mean", "GraphSage_maxpool", "GAT"]

In [258]:
gcn_model = train_model(300, "GCN", data_train, data_test, train_loader, test_loader, device)
test_model(gcn_model, data_total, data_test, device)

Epoch: 000, Loss: 1.3931, Train AUC: 0.6552, Test AUC: 0.6384
Epoch: 010, Loss: 1.1563, Train AUC: 0.8224, Test AUC: 0.8056
Epoch: 020, Loss: 0.7782, Train AUC: 0.9144, Test AUC: 0.8694
Epoch: 030, Loss: 0.7019, Train AUC: 0.9283, Test AUC: 0.8967
Epoch: 040, Loss: 0.6598, Train AUC: 0.9335, Test AUC: 0.9117
Epoch: 050, Loss: 0.6319, Train AUC: 0.9386, Test AUC: 0.9203
Epoch: 060, Loss: 0.5937, Train AUC: 0.9461, Test AUC: 0.9299
Epoch: 070, Loss: 0.6358, Train AUC: 0.9538, Test AUC: 0.9387
Epoch: 080, Loss: 0.5493, Train AUC: 0.9589, Test AUC: 0.9422
Epoch: 090, Loss: 0.5021, Train AUC: 0.9635, Test AUC: 0.9470
Epoch: 100, Loss: 0.4685, Train AUC: 0.9681, Test AUC: 0.9525
Epoch: 110, Loss: 0.4368, Train AUC: 0.9713, Test AUC: 0.9540
Epoch: 120, Loss: 0.4151, Train AUC: 0.9735, Test AUC: 0.9572
Epoch: 130, Loss: 0.3914, Train AUC: 0.9761, Test AUC: 0.9583
Epoch: 140, Loss: 0.4179, Train AUC: 0.9782, Test AUC: 0.9586
Epoch: 150, Loss: 0.4174, Train AUC: 0.9736, Test AUC: 0.9551
Epoch: 1

In [259]:
graphsage_mean_model = train_model(300, "GraphSage_mean", data_train, data_test, train_loader, test_loader, device)
test_model(graphsage_mean_model, data_total, data_test, device)

Epoch: 000, Loss: 1.3869, Train AUC: 0.7256, Test AUC: 0.6856
Epoch: 010, Loss: 1.0177, Train AUC: 0.8630, Test AUC: 0.8316
Epoch: 020, Loss: 0.7404, Train AUC: 0.9197, Test AUC: 0.8906
Epoch: 030, Loss: 0.6762, Train AUC: 0.9320, Test AUC: 0.9113
Epoch: 040, Loss: 0.6283, Train AUC: 0.9406, Test AUC: 0.9255
Epoch: 050, Loss: 0.5672, Train AUC: 0.9553, Test AUC: 0.9379
Epoch: 060, Loss: 0.5643, Train AUC: 0.9565, Test AUC: 0.9385
Epoch: 070, Loss: 0.5164, Train AUC: 0.9621, Test AUC: 0.9448
Epoch: 080, Loss: 0.4787, Train AUC: 0.9668, Test AUC: 0.9472
Epoch: 090, Loss: 0.4485, Train AUC: 0.9706, Test AUC: 0.9486
Epoch: 100, Loss: 0.4204, Train AUC: 0.9737, Test AUC: 0.9502
Epoch: 110, Loss: 0.4207, Train AUC: 0.9714, Test AUC: 0.9351
Epoch: 120, Loss: 0.3986, Train AUC: 0.9754, Test AUC: 0.9521
Epoch: 130, Loss: 0.3785, Train AUC: 0.9780, Test AUC: 0.9549
Epoch: 140, Loss: 0.3658, Train AUC: 0.9795, Test AUC: 0.9571
Epoch: 150, Loss: 0.3560, Train AUC: 0.9806, Test AUC: 0.9558
Epoch: 1

In [260]:
graphsage_max_model = train_model(300, "GraphSage_maxpool", data_train, data_test, train_loader, test_loader, device)
test_model(graphsage_max_model, data_total, data_test, device)

Epoch: 000, Loss: 1.3876, Train AUC: 0.6412, Test AUC: 0.6055
Epoch: 010, Loss: 1.1422, Train AUC: 0.8642, Test AUC: 0.8461
Epoch: 020, Loss: 0.7769, Train AUC: 0.9144, Test AUC: 0.9047
Epoch: 030, Loss: 0.7124, Train AUC: 0.9278, Test AUC: 0.9182
Epoch: 040, Loss: 0.6757, Train AUC: 0.9329, Test AUC: 0.9222
Epoch: 050, Loss: 0.6503, Train AUC: 0.9365, Test AUC: 0.9243
Epoch: 060, Loss: 0.6074, Train AUC: 0.9457, Test AUC: 0.9286
Epoch: 070, Loss: 0.5693, Train AUC: 0.9571, Test AUC: 0.9380
Epoch: 080, Loss: 0.5054, Train AUC: 0.9633, Test AUC: 0.9400
Epoch: 090, Loss: 0.4662, Train AUC: 0.9678, Test AUC: 0.9458
Epoch: 100, Loss: 0.4436, Train AUC: 0.9706, Test AUC: 0.9489
Epoch: 110, Loss: 0.4286, Train AUC: 0.9733, Test AUC: 0.9520
Epoch: 120, Loss: 0.4047, Train AUC: 0.9761, Test AUC: 0.9526
Epoch: 130, Loss: 0.4053, Train AUC: 0.9735, Test AUC: 0.9572
Epoch: 140, Loss: 0.3859, Train AUC: 0.9772, Test AUC: 0.9554
Epoch: 150, Loss: 0.3673, Train AUC: 0.9799, Test AUC: 0.9577
Epoch: 1

In [261]:
gat_model = train_model(300, "GAT", data_train, data_test, train_loader, test_loader, device)
test_model(gat_model, data_total, data_test, device)

Epoch: 000, Loss: 1.4268, Train AUC: 0.6212, Test AUC: 0.6517
Epoch: 010, Loss: 1.2028, Train AUC: 0.7680, Test AUC: 0.7616
Epoch: 020, Loss: 1.0571, Train AUC: 0.8203, Test AUC: 0.8195
Epoch: 030, Loss: 0.9470, Train AUC: 0.8656, Test AUC: 0.8682
Epoch: 040, Loss: 0.8320, Train AUC: 0.8988, Test AUC: 0.8980
Epoch: 050, Loss: 0.7564, Train AUC: 0.9176, Test AUC: 0.9114
Epoch: 060, Loss: 0.7056, Train AUC: 0.9281, Test AUC: 0.9174
Epoch: 070, Loss: 0.6844, Train AUC: 0.9317, Test AUC: 0.9227
Epoch: 080, Loss: 0.6683, Train AUC: 0.9344, Test AUC: 0.9252
Epoch: 090, Loss: 0.6610, Train AUC: 0.9379, Test AUC: 0.9376
Epoch: 100, Loss: 0.6533, Train AUC: 0.9387, Test AUC: 0.9366
Epoch: 110, Loss: 0.7007, Train AUC: 0.9388, Test AUC: 0.9370
Epoch: 120, Loss: 0.6487, Train AUC: 0.9391, Test AUC: 0.9394
Epoch: 130, Loss: 0.6417, Train AUC: 0.9416, Test AUC: 0.9409
Epoch: 140, Loss: 0.6294, Train AUC: 0.9434, Test AUC: 0.9428
Epoch: 150, Loss: 0.6172, Train AUC: 0.9449, Test AUC: 0.9424
Epoch: 1

# Application: 6 vs 6 win strategy (ordering)


Implementing an application that dynamically suggests the Pokemon you should send out to win when the order of the opponent's Pokemon appears randomly.

In [262]:
def node_feature_update(winner, loser, data_battle):
  """
  Update the node feature of the winner and loser pokemon after the battle

  INPUT
  winner: winner pokemon id
  loser: loser pokemon id
  data_battle: data object for the battle

  """
  damage = sum(data_battle.x[loser][i].item() for i in range(2, 8))
  data_battle.x[winner][1] -= damage
  data_battle.x[winner][0] = sum(data_battle.x[winner][i].item() for i in range(1, 8))

  return data_battle


def find_pokemon(model, opponents, ours, opp_idx, ordering, data_battle):
    """
    Find the pokemon to fight against the opponent pokemon
    If there is no pokemon that can win, choose the pokemon with the lowest stats among them and update the node feature
    If there are pokemons that can win, choose the pokemon with the lowest probability of winning among them and update the node feature

    INPUT
    model: gcn model, graphsage_mean model, graphsage_max_model, gat_model
    opponents: list of opponent pokemon id
    ours: list of our alive pokemon id
    opp_idx: index of the opponent pokemon in the opponents list
    ordering: list of pokemon id that have already been selected for the battle
    data_battle: data object for the battle


    OUTPUT
    win: 1 if we lose, 2 if we win (0: default)
    data_battle: updated data_battle matrix
    my_pokemon: my pokemon participating in this round
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    win = 0
    node1 = opponents[opp_idx]
    my_winner_pokemon = [] # (pokemon_id, win_prob) list

    for our_idx in range(len(ours)):
      node2 = ours[our_idx]
      if node2 in ordering: continue
      prediction, reverse_prediction = predict(model, node1, node2, data_battle.to(device), device) # 우리 pokemon이 이길 prediction score
      if prediction > reverse_prediction:
        my_winner_pokemon.append((ours[our_idx], prediction))

    if not len(my_winner_pokemon):
      # 이길 수 있는 pokemon이 없다면
      sorted_stats_idx = sorted(range(len(ours)), key=lambda i: data_battle.x[ours[i]][0].item())
      for i in range(6):
        if ours[sorted_stats_idx[i]] not in ordering:
          pokemon_idx = sorted_stats_idx[i]
          break
      my_pokemon = ours[pokemon_idx]
      data_battle = node_feature_update(opponents[opp_idx], ours[pokemon_idx], data_battle)
      win = 1 # we lose
    else:
      # 이길 수 있는 pokemon이 있다면 -> prob 크기 순서대로 정렬하고 prob 가장 작은 pokemon을 ordering에 추가
      my_winner_pokemon = sorted(my_winner_pokemon, key=lambda pokemon_pair: pokemon_pair[1]) # prediction score 값을 토대로 sorting
      winner = my_winner_pokemon[0][0]
      my_pokemon = winner
      data_battle = node_feature_update(winner, opponents[opp_idx], data_battle)
      win = 2 # we win

    return win, data_battle, my_pokemon

def search_pokemon_name(pokemon_id):
  for i in range(len(pokemon_data)):
    if pokemon_data.loc[i]["#"]==pokemon_id:
      return pokemon_data.loc[i]["Name"]

def print_simulate_msg(win, opponent, our):
  opponent_pokemon_name = search_pokemon_name(opponent)
  our_pokemon_name = search_pokemon_name(our)
  print("opponent: {} ({})".format(opponent, opponent_pokemon_name))
  print("our: {} ({})".format(our, our_pokemon_name))
  if (win ==1):
    # opponent win
    print("winner: {}".format(opponent))
  else:
    # our win
    print("winner: {}".format(our))
  print("")

def update_remained_pokemon(win, my_pokemon, opp_idx, opponent_pokemon, remained_opponents, remained_ours):
  # 패배한 포켓몬은 remained list에서 제거
  if (win == 1):
    # opponent win -> our pokemon이 제거되어야 한다
    remained_ours.remove(my_pokemon)
  else:
    # we win -> opponent pokemon이 제거되어야 한다.
    remained_opponents.remove(opponents[opp_idx])
    opp_idx += 1 # battle 할 다음 opponent 포켓몬을 indexing 하도록 한다.
  return remained_opponents, remained_ours, opp_idx

In [263]:
# model input: data, edge_index, neg_edge_index
# edge index를 생성하여 predict를 하자.
# node 1(head): loser, node 2(tail): winner이라는 것에 대해서 추측하는 것

# 6마리의 pokemon을 받는다.
# edge_index: [[losers], [winners]]인데 그냥 [[opponent's pokemons], [our pokemons]]로 넣고 edge prediction 진행
def simulate(model, opponents, ours, data_battle):
  """
  Simulate the battle between our 6 pokemons and opponent's 6 pokemons

  INPUT
  model: gcn model, graphsage_mean model, graphsage_max_model, gat_model
  opponents: list of opponent's 6 pokemon ids
  ours: list of our 6 pokemon ids
  data_battle: data object for the battle
  """
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  ordering = []
  random.shuffle(opponents) # opponent
  win = 0 # 0: start, 1: opponent win , 2: ours win
  opp_idx, our_idx = 0, 0
  remained_opponents, remained_ours = copy.deepcopy(opponents), ours
  round = 1 # round #

  while (len(remained_opponents)!=0 and len(remained_ours)!=0): # ours 또는 opponents 중 하나가 0이 될 때까지 repeat
    print("Round {}.".format(round))
    my_winner_pokemon = [] # (pokemon_id, win_prob) list to pick best our pokemon in this round

    win, data_battle, my_pokemon = find_pokemon(model, opponents, remained_ours, opp_idx, ordering, data_battle)
    print_simulate_msg(win, opponents[opp_idx], my_pokemon)
    remained_opponents, remained_ours, opp_idx = update_remained_pokemon(win, my_pokemon, opp_idx, opponents[opp_idx], remained_opponents, remained_ours)

    round += 1

  if (len(remained_ours)==0):
    print("You lose...")

  else:
    print("You win!")


In [265]:

opponents = [132, 155, 610, 382, 100, 519]
ours = [718, 357, 775, 356, 123, 635]


model = [gcn_model, graphsage_mean_model, graphsage_max_model, gat_model]
data_battle = copy.deepcopy(data_total)
simulate(model[3], opponents, ours, data_battle)

Round 1.
opponent: 132 (Scyther)
our: 635 (Gothita)
winner: 132

Round 2.
opponent: 132 (Scyther)
our: 356 (Spoink)
winner: 132

Round 3.
opponent: 132 (Scyther)
our: 718 (Chespin)
winner: 718

Round 4.
opponent: 100 (Haunter)
our: 718 (Chespin)
winner: 100

Round 5.
opponent: 100 (Haunter)
our: 357 (Grumpig)
winner: 100

Round 6.
opponent: 100 (Haunter)
our: 775 (Sliggoo)
winner: 775

Round 7.
opponent: 155 (Snorlax)
our: 123 (Kangaskhan)
winner: 123

Round 8.
opponent: 519 (Togekiss)
our: 123 (Kangaskhan)
winner: 123

Round 9.
opponent: 610 (Basculin)
our: 123 (Kangaskhan)
winner: 610

Round 10.
opponent: 610 (Basculin)
our: 775 (Sliggoo)
winner: 610

You lose...
